In [ ]:

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import pickle
from keras.models import Sequential
from keras import layers

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from prediction_reactions.service.preprocessing_data.prepearing_data import resources


In [ ]:
encoder = LabelEncoder()
df=pd.read_csv(resources+"data/"+"all_data_Category1.csv",names=['label','data'])
df=df.dropna()
# df_yelp = df[df['source'] == 'yelp']
sentences = df['data'].values
y =encoder.fit_transform(df['label'].values)
sentences_train, sentences_test, y_train, y_test = train_test_split(sentences, y, test_size=0.25, random_state=1000)

In [ ]:
from sklearn.ensemble import RandomForestClassifier 
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(sentences_train)

X_train = tokenizer.texts_to_sequences(sentences_train)
X_test = tokenizer.texts_to_sequences(sentences_test)

vocab_size = len(tokenizer.word_index) + 1  # Adding 1 because of reserved 0 index

print('vocab size',vocab_size)
print(sentences_train[2])
print(X_train[2])

In [ ]:
maxlen = 100
X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

In [ ]:
encoders = OneHotEncoder(sparse=False)
Y_train=encoders.fit_transform(y_train.reshape(-1,1))
Y_test=encoders.transform(y_test.reshape(-1,1))


In [ ]:
embedding_dim = 300
# layers.bi
model = Sequential()
model.add(layers.Embedding(input_dim=5000, 
                           output_dim=embedding_dim, 
                           input_length=100))
model.add(layers.Bidirectional(layers.LSTM(units=300)))
model.add(layers.Flatten())
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(12, activation='softmax'))
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

In [ ]:
model.fit(X_train,Y_train,epochs=2)

In [ ]:
model.save(resources+"models/"+"model_category.h5")

In [ ]:
pickle.dump(tokenizer,open(resources+"models/"+"tokenizer_model_category_pickle","wb"))

In [ ]:
pickle.dump(encoder,open(resources+"models/"+"encoder_model_category_pickle","wb"))